In [1]:
from premise import *
import bw2data
import bw2io as bi
from datapackage import Package
import premise
import pandas as pd

# Brightway packages
import bw2io as bi 
import bw2data as bd
import bw2calc as bc
# Brightway type hints
from bw2io import SingleOutputEcospold2Importer
# import/export
import os

# Now import shared variables
from config import *
from private_keys import *

#Standard vars
bw2data.projects.set_current(PROJECT_NAME)
premise.__version__,PROJECT_NAME

((2, 3, 0, 'dev0'), 'bw25')

### 1. Here we decide whether we want to generate a new brightway database or export the databases directly as efficient data packages

In [2]:
create_db_package = True
create_dbs = False

### 1.1 Decide on the scenarios to be included for this exercise:
* A business-as-usual scenario, using current trends from output energy system scenarios from OPEN-PROM, EDM-I, FORECAST, and combine without circular economy measures.
* A low-carbon scenario, using low-carbon output energy system scenarios from OPEN-PROM, EDM-I, FORECAST, and combine it to ambitious circular economy measures.

All information is provided in data packages, we import it and define the scenarios below

In [3]:
sps_op = Package(r"datapackage_open_prom.json")
sps_edm_i = Package(r"datapackage_edm_i.json")
sps_ce = Package(r"datapackage_ce.json")
sps_forecast = Package(r"datapackage_forecast.json")
bau_iam_scenario = 'SSP2-Base'#"SSP2-PkBudg1150"

#1. OPEN-PROM file with all energy system info
#2. modifying EU steel sector.
#3. modifying activities to consider CE measures --> change lifetimes, specific use of materials, and recycled shares.
#4. OPTIONAL: modifying ammonia and methanol markets for all EU-27 countries.

# Low-carbon scenario:
scenarios = [
        {"model": "remind", 'year': 2040, "pathway": "SSP2-PkBudg1150", "external scenarios": [
                                                                            {"scenario": "WP1 NetZero", "data": sps_op}, #OPEN-PROM
                                                                            {"scenario": "CBAM", "data": sps_edm_i},  #EDM-invest
                                                                            {"scenario": "WP1 NetZero_ce", "data": sps_ce}, #Additional CE measures
                                                                            {"scenario": "fc_nz", "data": sps_forecast}, #FORECAST is focused on low-carbon scenario
                                                                                ] },
    #Test whether there is difference
     #   {"model": "remind", 'year': 2040, "pathway": bau_iam_scenario, "external scenarios": [
     #                                                                       {"scenario": "WP1 NPI", "data": sps_op}, #OPEN-PROM
     #                                                                       {"scenario": "INDEPENDENCE", "data": sps_edm_i}, #EDM-invest
     #                                                                       {"scenario": "WP1 NPI_ce", "data": sps_ce},#Additional CE measures
     #                                                                           ] },
#BAU:
        {"model": "remind", 'year': 2040, "pathway": bau_iam_scenario, "external scenarios": [
                                                                          {"scenario": "WP1 NPI", "data": sps_op}, #OPEN-PROM
                                                                         {"scenario": "INDEPENDENCE", "data": sps_edm_i}, #EDM-invest
                                                                            {"scenario": "WP1 NPI_ce", "data": sps_ce},#Additional CE measures
                                                                              ] }

]

In [4]:
for db in list(bi.databases):
    if db == 'ecoinvent-3.10-cutoff':
        pass
        #del bd.databases['ecoinvent-3.10-cutoff']
list(bi.databases)

['ecoinvent-3.10-biosphere',
 'ecoinvent-3.10-cutoff',
 'remind_2040_SSP2-PkBudg1150_WP1-NetZero_CBAM_WP1-NetZero_ce_fc_nz',
 'remind_2040_SSP2-Base_WP1-NPI_INDEPENDENCE_WP1-NPI_ce']

### 1.2 Now load the ecoinvent database using brightway 2.5

In [5]:
# Create dbs if not existing yet
if DB_NAME_INIT not in bi.databases:
    bi.import_ecoinvent_release("3.10", "cutoff", USER_NAME, USER_PW)
bi.databases

Databases dictionary with 4 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	remind_2040_SSP2-Base_WP1-NPI_INDEPENDENCE_WP1-NPI_ce
	remind_2040_SSP2-PkBudg1150_WP1-NetZero_CBAM_WP1-NetZero_ce_fc_nz

### 1.3 Now, we will ahve to reformat the output file from EDM-Invest to comply with the IAMC format of premise, if needed

In [6]:
def reformat_csv_open_prom(input_file, output_file, sheet_name='together'):#'data(2)'
    """
    Reformats an input Excel file containing the EMD-I steel model data and saves it in a new CSV format.

    This function performs the following operations:
    - Reads data from an Excel file.
    - Replaces specified region names using a predefined mapping dictionary.
    - Selects relevant columns for the output.
    - Renames columns to match the desired format.
    - Saves the reformatted data into a CSV file with formatted floating-point values.

    Parameters:
    -----------
    input_file : str
        The path to the input Excel file. This file should contain the raw data, including columns
        for 'model', 'scenario', 'region', 'variable', 'unit', and yearly data (e.g., '2020', '2025', etc.).
    
    output_file : str
        The path to the output CSV file where the reformatted data will be saved. The file will be saved with
        formatted floating-point values for the year columns (e.g., 2020, 2025, etc.) rounded to two decimal places.

    Returns:
    --------
    None
        This function does not return anything. The reformatted data is written to the `output_file`.

    Notes:
    ------
    - The region column values can be customized via the `region_replacements` dictionary. By default, 
      'EU27' is replaced with 'RER'. You can add more region mappings to the dictionary as needed.
    - The function reads data from an Excel file using `pd.read_excel()`. Ensure the input file is in the correct format.
    - The column names in the output are adjusted to match the expected format ('variables' instead of 'variable').

    Example:
    --------
    input_file = 'data/input_file.xlsx'
    output_file = 'data/output_file.csv'
    reformat_csv_edm_i(input_file, output_file)
    
    This will read the data from 'input_file.xlsx', apply the region replacements, select relevant columns,
    rename them, and save the result to 'output_file.csv'.
    """
    # Read the original CSV file
    df = pd.read_excel(input_file, sheet_name=sheet_name)
    df.columns = df.columns.astype(str)
    print(df.columns)

    # Select only the required columns and rename them
    df = df[['Model', 'Scenario', 'Region', 'Variable', 'Unit', '2020', '2025', '2030', '2035', '2040', '2045', '2050']]
    
    # Rename columns to match the desired output format
    df.columns = ['model', 'scenario', 'region', 'variables', 'unit', '2020', '2025', '2030', '2035', '2040', '2045', '2050']

    # Define the dictionary for region replacements
    region_replacements = {
        'Europe': 'RER',  # Replace 'NewRegion1' with the desired region name
        # Add more mappings as needed, e.g., 'OldRegion': 'NewRegion'
    }
    df['region'] = df['region'].replace(region_replacements)

    unit_replacements = {
        'Mt Crude steel': 'Mtonne/yr',  # Replace 'NewRegion1' with the desired region name
        'Mt CO2/yr': 'Mtonne/yr',  # Replace 'NewRegion1' with the desired region name
        # Add more mappings as needed, e.g., 'OldRegion': 'NewRegion'
    }
    df['unit'] = df['unit'].replace(unit_replacements)

    scenario_replacements = {
        #'WP1 NPI': 'NPI', 
        #'WP1 NetZero': 'NETZERO',
    }

    df['scenario'] = df['scenario'].replace(scenario_replacements)
    df.fillna(0, inplace=True)
    
    # Save the reformatted DataFrame to a new CSV file
    df.to_csv(output_file, index=False, float_format="%.2f")

# Usage
input_file_edm = r"scenario_data\OPEN-PROM-current-version (11.24)-report-NPi-NZ.xlsx"
output_file = "scenario_data\scenario_data_open_prom.csv"

if create_db_package:
    pass
    #reformat_csv_open_prom(input_file_edm, output_file)

In [7]:
def reformat_csv_edm_i(input_file, output_file):
    """
    Reformats an input Excel file containing the EMD-I steel model data and saves it in a new CSV format.

    This function performs the following operations:
    - Reads data from an Excel file.
    - Replaces specified region names using a predefined mapping dictionary.
    - Selects relevant columns for the output.
    - Renames columns to match the desired format.
    - Saves the reformatted data into a CSV file with formatted floating-point values.

    Parameters:
    -----------
    input_file : str
        The path to the input Excel file. This file should contain the raw data, including columns
        for 'model', 'scenario', 'region', 'variable', 'unit', and yearly data (e.g., '2020', '2025', etc.).
    
    output_file : str
        The path to the output CSV file where the reformatted data will be saved. The file will be saved with
        formatted floating-point values for the year columns (e.g., 2020, 2025, etc.) rounded to two decimal places.

    Returns:
    --------
    None
        This function does not return anything. The reformatted data is written to the `output_file`.

    Notes:
    ------
    - The region column values can be customized via the `region_replacements` dictionary. By default, 
      'EU27' is replaced with 'RER'. You can add more region mappings to the dictionary as needed.
    - The function reads data from an Excel file using `pd.read_excel()`. Ensure the input file is in the correct format.
    - The column names in the output are adjusted to match the expected format ('variables' instead of 'variable').

    Example:
    --------
    input_file = 'data/input_file.xlsx'
    output_file = 'data/output_file.csv'
    reformat_csv_edm_i(input_file, output_file)
    
    This will read the data from 'input_file.xlsx', apply the region replacements, select relevant columns,
    rename them, and save the result to 'output_file.csv'.
    """
    # Read the original CSV file
    df = pd.read_excel(input_file)

    # Define the dictionary for region replacements
    region_replacements = {
        'EU27': 'RER',  # Replace 'NewRegion1' with the desired region name
        # Add more mappings as needed, e.g., 'OldRegion': 'NewRegion'
    }
    df['region'] = df['region'].replace(region_replacements)

    unit_replacements = {
        ' Mt Crude steel': 'Mtonne/yr',  # Replace 'NewRegion1' with the desired region name
        # Add more mappings as needed, e.g., 'OldRegion': 'NewRegion'
    }
    df['unit'] = df['unit'].replace(unit_replacements)

    # Select only the required columns and rename them
    df = df[['model', 'scenario', 'region', 'variable', 'unit', '2020', '2025', '2030', '2035', '2040', '2045', '2050']]
    
    # Rename columns to match the desired output format
    df.columns = ['model', 'scenario', 'region', 'variables', 'unit', '2020', '2025', '2030', '2035', '2040', '2045', '2050']
    
    # Save the reformatted DataFrame to a new CSV file
    df.to_csv(output_file, index=False, float_format="%.2f")

# Usage
input_file_edm = r"scenario_data\PRM-1_Study4_EU-Steel_Results_EDM-I.xlsx"
output_file = "scenario_data\scenario_data_edm_i.csv"
#reformat_csv_edm_i(input_file_edm, output_file)

In [8]:
# check for NaNs
df_check = pd.read_csv(r"scenario_data\scenario_data_forecast_all.csv")
df_check.isna().sum()

model        0
scenario     0
region       0
variables    0
unit         0
2020         0
2025         0
2030         0
2035         0
2040         0
2045         0
2050         0
dtype: int64

### 1.4 if we want to export it as a brightway database, we can do this as follows:

In [9]:
import re
def clean(s: str) -> str:
    """Replace spaces with - and ensure it's filename safe."""
    return re.sub(r"\s+", "-", s)

scenario_names = []

for sc in scenarios:
    parts = [sc["model"], str(sc["year"]), sc["pathway"]]
    external = [clean(item["scenario"]) for item in sc["external scenarios"]]
    parts.extend(external)
    scenario_names.append("_".join(parts))
scenario_names

['remind_2040_SSP2-PkBudg1150_WP1-NetZero_CBAM_WP1-NetZero_ce_fc_nz',
 'remind_2040_SSP2-Base_WP1-NPI_INDEPENDENCE_WP1-NPI_ce']

In [10]:
if create_dbs:
    clear_cache()
    ndb = NewDatabase(
        scenarios = scenarios,
        source_db=DB_NAME_INIT, # <-- name of the database in the BW2 project. Must be a string.
        source_version=EI_VERSION, # <-- version of ecoinvent. Must be a string.
        key=KEY_PREMISE, # <-- ask the key to run premise from the developers
        use_absolute_efficiency=True,
        biosphere_name=BIOSPHERE_DB)

    ndb.update()
    
    ndb.write_db_to_brightway(name=scenario_names)

### 1.4.1 If we want to evaluate this just exported brightway database, we can check some activities as follows and calculate their LCA score.
For example, we could calculate the LCA impacts on climate change for the following new market activities generated:
* Methanol (based on FORECAST)
* Ammonia (based on FORECAST)
* Electricity market (based on OPEN-PROM)
* Steel (Based on EDM-I)

In [11]:
db_select = scenario_names[0]

if create_dbs:
    def calculate_lca_for_activity(db_name, activity_name, ref_product, location, impact_method, amount_fu=1):
        """
        Calculate the Life Cycle Assessment (LCA) score for a specified activity.
    
        Parameters:
            db_name (str): Name of the Brightway2 database to search.
            activity_name (str): Name of the activity to calculate the LCA for.
            ref_product (str): Reference product of the activity.
            location (str): The geographical location of the activity.
            impact_method (tuple): The LCIA method tuple to calculate the environmental impact.
    
        Returns:
            float: The LCA score for the specified activity.
    
        Raises:
            IndexError: If the specified activity cannot be found in the database.
            ValueError: If the database name, method, or input parameters are invalid.
    
        Example:
            # Define the LCIA method
            cc_mt = [mt for mt in bd.methods if 'EF v3.1 EN' in str(mt) and 'climate change' in mt][0]
    
        """
    
        [act for act in bd.Database(db_select) if '(SPS)' in act['name']]
        # Retrieve the desired activity
        activity = [
            act for act in bd.Database(db_name)
            if activity_name == act['name'] and location == act['location'] and
            ref_product == act['reference product']][0]
    
        # Perform LCA calculation
        lca = bc.LCA({activity: amount_fu}, method=impact_method)
        lca.lci()
        lca.lcia()
        
        return lca.score
    
    cc_mt = [mt for mt in bd.methods if 'EF v3.1 EN' in str(mt) and 'climate change' in mt][0]
    
    acts = [
            ("market for methanol (SPS)", "methanol", "NL"),
            ("market for methanol (SPS)", "methanol", "RER"),
            ("market for ammonia (SPS)", 'ammonia, anhydrous, liquid', "NL"),
            ("market for ammonia (SPS)", 'ammonia, anhydrous, liquid', "RER"),
            ("market group for electricity, low voltage (SPS)", "electricity, low voltage", "RER"),
            ("market group for electricity, high voltage (SPS)", "electricity, high voltage", "RER"),
            ("market for steel, low-alloyed (SPS)", "steel, low-alloyed", "RER"),
            ("market for steel, unalloyed (SPS)", "steel, unalloyed", "RER"),
           ]
            
    for act in acts:
        score = calculate_lca_for_activity(
            db_name=db_select,
            activity_name=act[0],  # Replace with your activity
            ref_product=act[1],  # Replace with your reference product
            location=act[2],  # Replace with your location filter
            impact_method=cc_mt
        )
        
        print(f"LCA [{cc_mt[2]}] score for {act}: {round(score,3)}")

    act = [act for act in bd.Database(db_select) if 'methanol (SPS)' in act['name'] and act['location']=='NL'][0]
    for exc in act.exchanges():
        print(exc['name'], exc['location'], exc['amount'])

act = [act for act in bd.Database(db_select) if 'electricity, used in residential electric appliances' in act['name']][0] #and act['location']=='NL'][0]
for exc in act.exchanges():
    if exc['type']=='technosphere':
        print(exc['name'], exc['location'], exc['amount'])

### 2. Export the defined scenarios for chosen years as data packages needed for PATHWAYS in the next step (see the other Notebook `2_calc_impacts_ALL.ipynb`)

In [12]:
for db in list(bi.databases):
    if 'ei_cutoff_3.10_remind' in str(db):
        print(db)
        #del bi.databases[db]

scenarios

In [ ]:
if create_db_package:
    #if 'external scenarios' in str(scenario):
    #names=[f"{scenario['model']}-{scenario['pathway']}-edm_i-{scenario['external scenarios'][0]['scenario']}" for scenario in scenarios]
    ndb = PathwaysDataPackage(
        scenarios=scenarios,
        #years=[2050],
        years=[2025, 2030, 2035, 2040, 2045, 2050],#, 2050],
        source_db=DB_NAME_INIT,
        source_version=EI_VERSION,
        key=KEY_PREMISE,
        use_absolute_efficiency=True,
        biosphere_name=BIOSPHERE_DB )

    ndb.create_datapackage(
        name=f"remind-transience",
        contributors=[
            {"name": "Tom Terlouw",
            "email": "tom.terlouw@psi.ch"}] )

premise v.(2, 3, 0, 'dev0')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

Processing scenarios for all sectors:   0%|    | 0/12 [00:00<?, ?it/s]

Extracted 1 worksheets in 0.82 seconds
Extracted 1 worksheets in 0.08 seconds
Extracted 1 worksheets in 0.04 seconds
Extracted 1 worksheets in 0.08 seconds
Amount adapted in battery production, Li-ion, NMC811, rechargeable, prismatic for market for copper, anode from 0.001 to 0.0005, and new copper scrap, sorted, pressed, Recycled Content cut-off exchange added.
Amount adapted in battery production, Li-ion, NCA, rechargeable, prismatic for market for copper, anode from 0.00061 to 0.0003, and new copper scrap, sorted, pressed, Recycled Content cut-off exchange added.
Amount adapted in battery production, Li-ion, NMC111, rechargeable, prismatic for market for copper, anode from 0.00055 to 0.00028, and new copper scrap, sorted, pressed, Recycled Content cut-off exchange added.
Amount adapted in battery production, Li-ion, NCA, rechargeable, prismatic for market for copper, anode from 0.00061 to 0.0003, and new copper scrap, sorted, pressed, Recycled Content cut-off exchange added.
Amount 

Processing scenarios for all sectors:   8%| | 1/12 [13:13<2:25:30, 793

### 3. Continue in Notebook `2_calc_impacts.ipynb` to calculate environmental burdens of sectors or entire energy system scenarios using the exported data packages from the previous step